In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

import os
import sys

project_pth = os.path.join(os.getcwd(), '..', '..')
sys.path.append(project_pth)

from utils.transformations import reusable


In [0]:
df= spark.read.format("parquet")\
    .load("abfss://broze@storageazureprojectnimit.dfs.core.windows.net/DimUser")



In [0]:
display(df)

#### **AUTOLOADER**

In [0]:

df_user = (
    spark.readStream.format("cloudFiles")
        .option("cloudFiles.format", "parquet")
        .option(
            "cloudFiles.schemaLocation",
            "abfss://silver@storageazureprojectnimit.dfs.core.windows.net/DimUser/checkpoint/"
        )
        .load("abfss://broze@storageazureprojectnimit.dfs.core.windows.net/DimUser/")
)


In [0]:
display(df_user)

In [0]:
df_user = df_user.withColumn("user_name", upper(col("user_name")))
display(df_user)

In [0]:
df_user_obj = reusable()

df_user = df_user_obj.dropcolumns(df_user, ['_rescued_data'])
df_user = df_user.dropDuplicates(['user_id'])
display(df_user)


#### **DimUser**

In [0]:
df_user.writeStream.format("delta") \
    .outputMode("append") \
    .option("checkpointLocation", "abfss://silver@storageazureprojectnimit.dfs.core.windows.net/DimUser/checkpoint") \
    .trigger(once=True) \
    .start("abfss://silver@storageazureprojectnimit.dfs.core.windows.net/DimUser/data")


#### **Dim Artist**

In [0]:
df_art = (
    spark.readStream.format("cloudFiles")
        .option("cloudFiles.format", "parquet")
        .option(
            "cloudFiles.schemaLocation",
            "abfss://silver@storageazureprojectnimit.dfs.core.windows.net/DimArt/checkpoint/"
        )
        .load("abfss://broze@storageazureprojectnimit.dfs.core.windows.net/DimArt/")
)


In [0]:
display(df_art)

In [0]:
df_art_obj = reusable()

df_art = df_art_obj.dropcolumns(df_art, ['_rescued_data'])
df_art = df_art.dropDuplicates(['artist_id'])
display(df_art)


In [0]:
df_art.writeStream.format("delta")\
    .outputMode("append")\
    .option("checkpointLocation", "abfss://silver@storageazureprojectnimit.dfs.core.windows.net/DimArt/checkpoint")\
    .trigger(once=True)\
    .option("path", "abfss://silver@storageazureprojectnimit.dfs.core.windows.net/DimArt/data")\
    .toTable("spotify_cata.silver.DimArtist")
